In [67]:
import pandas as pd
import numpy as np
import math as ma
from sklearn import datasets

In [68]:
from sklearn import datasets
iris=datasets.load_iris()
x=pd.DataFrame(iris.data)             #x is dataframe of iris.data
x.columns=['sl','sw','pl','pw'] 
y=pd.DataFrame(iris.target)           #y is dataframe of iris.target

features=['sl','sw','pl','pw']       # list of features over which spliting will be done
#x.head()
#y.head()
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [69]:
#count the number of flower of each type
def count(output,typeflower):          
    output=np.array(output[:])
    return (output==typeflower).sum()

In [70]:
def entropy(lst):      #funtion to calculate entropy/information of given node
    info=0
    for i in range(3):
        prob=lst[i]/sum(lst)
        if prob==0:        # to handel log(0) which is undefine
            continue
        info+=-1*prob*ma.log2(prob)       
    return info

In [71]:
# this funtion finally retun gain ratio
def gain_ratio(lst,lst1,lst2):   
       
    info=entropy(lst)         #info is entropy of head node
    info1=entropy(lst1)        #info1 is entropy of first splited node
    info2=entropy(lst2)         # info2 is entropy of second splited node

    
   #info_gain=info-(a+b)  so we need to calculate a and b

    a=(sum(lst1)/(sum(lst1)+sum(lst2)))*info1     
    b=(sum(lst2)/(sum(lst1)+sum(lst2)))*info2   
    info_gain=info-(a+b)
    
    
    # calculation splitinfo of first splited node
    split1=((-1)*sum(lst1)/(sum(lst1)+sum(lst2)))*ma.log2(sum(lst1)/(sum(lst1)+sum(lst2)))
    
        
    # calculation splitinfo of second splited node
    split2=((-1)*sum(lst2)/(sum(lst1)+sum(lst2)))*ma.log2(sum(lst2)/(sum(lst1)+sum(lst2)))
        
    split_info=split1+split2
    gain_rati=info_gain/split_info      
  
    return gain_rati

In [72]:
def gain(x,y,f):     
    
    data=x[f]       
   
    
    maxx=0                   #maxx will give  max_gain ratio 
    minvalue=data.min()      # we are calculating min and max value to run loop over all values of data
    m=minvalue+0.1
    maxvalue=data.max()
    feat=0       # later feat will return this feature f in df function
    mid=0          # at each time mid will store the value at which spliting is done by feature f
    
    
    while(m<=maxvalue):    
        lst=[0,0,0]          #it will store number of 0`s , 1`s , 2`s of y on respective indexes 0,1,2
        lst1=[0,0,0]      #it will store number of 0`s , 1`s , 2`s of  split_1y on respective indexes 0,1,2
        lst2=[0,0,0]     #it will store number of 0`s , 1`s , 2`s of split_2y on respective indexes 0,1,2
        
        
        split_1x=x[data<m]         # it is split of x data whose values are less then m
        split_1y=y[data<m]          # it is split of y data whose values are less then m
        
        
        
        
        split_2x=x[data>=m]     # it is split of x data whose values are greater then m
        split_2y=y[data>=m]       # it is split of y data whose values are greater then m
        
        
        total_elements=len(x)   
        lst[0]=count(y,0)  
        lst[1]=count(y,1) 
        lst[2]=total_elements-lst[0]-lst[1]    
        
        total_elements=len(split_1x)                 
        lst1[0]=count(split_1y,0)
        lst1[1]=count(split_1y,1) 
        lst1[2]=total_elements-lst1[0]-lst1[1]
        
        total_elements=len(split_2x)    
        lst2[0]=count(split_2y,0)
        lst2[1]=count(split_2y,1) 
        lst2[2]=total_elements-lst2[0]-lst2[1]
       
        
        max_gain=gain_ratio(lst,lst1,lst2)    #gain_ratio fun will give u max gain ratio using all 3 list which have all data
        if max_gain>=maxx:
            maxx=max_gain
            feat=f
            mid=m
        m+=0.1       #incrementing m by 0.1 so that at every stage we can get best gain_ratio
    return maxx,feat,mid

In [78]:
def decesion_tree(x,y,features,level):
    
    lst=[0,0,0]                          #list contains the number of flowers of each type 
    no_of_features_left=len(features)
    total_elements=len(x)
    no_of_setosa=count(y,0)         #countSetosa is function to count number of setosa flowers in output
    no_of_versicolor=count(y,1) 
    no_of_virginica=count(y,2)
    lst[0]=no_of_setosa
    lst[1]=no_of_versicolor
    lst[2]=no_of_virginica
  
    print('level ',level)
    print('count of setosa =',no_of_setosa)
    print('count of versicolor =',no_of_versicolor)
    print('count of virginica =',no_of_virginica)
    print('current entropy is =',entropy(lst))
    print()
    
    #Base Case : Node is pure , i.e if lst has only one type of flowers it will reach leaf node
    
    if lst.count(0)==2:         
        print('Reached Leaf Node')
        return
    
    
    maxx=0                    # maxx will store the maxx gain ratio
    mid=0                     #mid is the value at which feature splits and gives max gain ratio
    feat=None
    
    
    #Finding best feat to split dataset
    
    for f in features:
        max_gain,final_feature,m=gain(x,y,f) 
        if max_gain>=maxx:
            maxx=max_gain
            feat=final_feature
            mid=m
            
    print('splitting on feature',feat,'with gain ratio',maxx)         #featuree is the feat at which split done
    
    new_1x=x[x[feat]<mid]     #spliting main data into two parts according to feat feature
   
    new_1y=y[x[feat]<mid]     #spliting main output into two parts according to feat feature
   
    
    new_2x=x[x[feat]>=mid]
    
    new_2y=y[x[feat]>=mid]
   
    
    # features will remain same bcz a feature can be used any number of times
    decesion_tree(new_1x,new_1y,features,level+1)    #calling dt again recursively
    decesion_tree(new_2x,new_2y,features,level+1)     #calling dt again recursively


In [80]:
#calling decesion tree function
decesion_tree(x,y,features,0)     #starting level=0

level  0
count of setosa = 50
count of versicolor = 50
count of virginica = 50
current entropy is = 1.584962500721156

splitting on feature pw with gain ratio 0.9999999999999999
level  1
count of setosa = 50
count of versicolor = 0
count of virginica = 0
current entropy is = 0.0

Reached Leaf Node
level  1
count of setosa = 0
count of versicolor = 50
count of virginica = 50
current entropy is = 1.0

splitting on feature pw with gain ratio 0.6933647985912662
level  2
count of setosa = 0
count of versicolor = 49
count of virginica = 5
current entropy is = 0.44506485705083865

splitting on feature pl with gain ratio 0.606617822020301
level  3
count of setosa = 0
count of versicolor = 49
count of virginica = 3
current entropy is = 0.31821529768323314

splitting on feature pl with gain ratio 0.2720453440631925
level  4
count of setosa = 0
count of versicolor = 47
count of virginica = 1
current entropy is = 0.1460942501201363

splitting on feature pw with gain ratio 1.0
level  5
count of set